# Dollar Cost Averaging (DCA) vs Lump Sum (LS)

> If you had 100K cash available in your bank and you are thinking of investing in a market index like the [S&P500](https://en.wikipedia.org/wiki/S%26P_500), then should you invest it 10K for the next 10 months or 100K today?

If you are thinking of this question, then congratulations! You are one step ahead than most who would simply jump on the bandwagon and follow mainstream advice. 

We can rarely be certain about one choice over another in practice but we can gain confidence if we apply a rigorous analytical approach to our decision. Here, we look to test two investment strategies, for a fixed amount of 100K on S&P500:

1. **Dollar Cost Averaging** (DCA)
2. **Lump Sum** (LS)

These strategies will be tested over the following parameters:

- **A large number of historical timepoints**: Such that the analysis can be generalized and answer probabilistic questions
- **Length of investment period**: 1, 3, 5, 10, 15, 20, 30 years
- **DCA periods**: What are the effects of investing, say 10K for 10 months vs 1K for 100 months (~8-9 years)?

## Setup

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import yfinance as yf

sns.set_theme(style="ticks", palette="pastel")
pd.options.display.float_format = "{:,.2f}".format

In [ ]:
# Read S&P historical adjusted prices
ticker = 'SPY'
sp500 = (
    yf
    .Ticker(ticker)
    .history(period = 'max')
    # .reset_index()
    .loc[:, ['Close']]
)

# Check data
display(sp500.head())
sp500.plot(title = 'S&P500 Adjusted Close Prices')
plt.show()

In [ ]:
# For backtest purposes, we fill non-working days with last prices
backtest_df = (    
    pd
    .DataFrame(index = pd.date_range(sp500.index.min(), sp500.index.max()))
    .rename_axis('Date')
    .merge(sp500, left_index = True, right_index = True, how = 'left')
    .ffill()
)

## Analyze

In [ ]:
# Parameters
timepoints = backtest_df.index.to_numpy()
starting_cash = 100000
bootstraps = 1000
investment_periods = [1, 3, 5, 10, 15, 20] # in years
dca_periods = np.arange(1, 5+1, 4)*12 # DCA periods for 1 year vs 5 years

In [ ]:
def invest(prices, invest_dates, amount):
    df = prices.copy()
    df.loc[lambda x: x.index.isin(invest_dates), 'invested'] = float(amount) / float(len(invest_dates)) # Invest equal amounts
    return df

def compute_returns(df, starting_cash):
    absolute_return = (df.invested * (1 + prices.returns)).sum() - starting_cash
    end_cash = starting_cash + absolute_return
    relative_return = (df.invested * (1 + prices.returns)).sum() / starting_cash - 1

    return [end_cash, absolute_return, relative_return]

In [ ]:
# Backtest
results = []
for investment_period in investment_periods:
    for _ in range(bootstraps):
        # Get key points
        max_start_point = timepoints.max() - pd.DateOffset(years = investment_period)
        possible_start_points = timepoints[timepoints <= max_start_point]
        start_point = np.random.choice(possible_start_points)
        end_point = start_point + pd.DateOffset(years = investment_period)

        # Get prices
        prices = (
            backtest_df
            .loc[lambda x: (x.index >= start_point) & (x.index <= end_point)]
            .assign(
                returns = lambda x: x.iloc[-1].Close / x.Close - 1,
                invested = 0.00,
            )
            .copy()
        )

        # Invest lump-sum
        lump_sum = invest(prices, [start_point], starting_cash)

        # Compute returns
        ls_returns = compute_returns(lump_sum, starting_cash)

        # Store results
        results.append([start_point, end_point, investment_period, 'LS', starting_cash,] + ls_returns)

        # Invest DCA
        dcas = []
        for dca_period in dca_periods:
            invest_dates = pd.date_range(start_point, start_point + pd.DateOffset(months = dca_period), freq = 'BME')
            dca = invest(prices, invest_dates, starting_cash)
            dca_returns = compute_returns(dca, starting_cash)
            results.append([start_point, end_point, investment_period, f'DCA{dca_period}', starting_cash, ] + dca_returns)

In [ ]:
# Store results
backtest = (
    pd
    .DataFrame(
        np.array(results), 
        columns = ['start_date', 'end_date', 'years', 'strategy', 'start_cash', 'end_cash', 'abs_return', 'pct_return'],
    )
)

## Evaluate

In [ ]:
# Compare by investment years and strategy
g = sns.catplot(data = backtest, x = 'strategy', y = 'pct_return', hue = 'strategy', col = 'years', aspect = 0.4)

# Set grid and y-axis to percentage
for i, ax in enumerate(g.axes.flatten()):
    ax.grid(visible = True, axis = 'y')
    if i == 0:
        labels = [f'{x:.0%}' for x in ax.get_yticks()]
        ax.set_yticklabels(labels)

plt.show()

In [ ]:
# Report statistics
confidence = 99
(
    backtest
    .groupby(['strategy', 'years'])
    .agg(
        {
            'pct_return': ['mean', 'min', 'max', lambda x: np.percentile(x, 100-confidence), lambda x: np.percentile(x, confidence)]
        }
    )
    .reset_index()
    .set_axis(['strategy', 'years', 'x_return', 'min', 'max', f'{100-confidence}%', f'{confidence}%'], axis = 1)
    .pivot_table(index = 'years', columns = 'strategy', values = ['x_return', '1%', '99%'])
    + 1
) * starting_cash